# Classificatie van nieuwsartikelen

In deze notebook gaan we verder werken op de AG-news nieuwsartikelen dataset.
In de vorige notebook hebben we bekeken hoe we tekstuele data kunnen preprocessen.
In deze notebook gaan we classificatie uitvoeren door gebruik te maken van recurrente neurale netwerken.

In [10]:
[5] * 0

[]

In [15]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import kagglehub
from sklearn.model_selection import train_test_split
from collections import Counter
import re

MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 60

# Load the dataset
path = kagglehub.dataset_download("amananandrai/ag-news-classification-dataset")

def read_csv(filepath):
    df = pd.read_csv(filepath)
    df['text'] = df['Title'] + ' ' + df['Description']
    df['label'] = df['Class Index'] - 1
    return df[['text', 'label']]

df_train = read_csv(f'{path}/train.csv')
df_test = read_csv(f'{path}/test.csv')

display(df_train.head())

# Tokenizer
def simple_tokenizer(text):
    return re.findall(r"\b\w+\b", text.lower())

# tel hoeveel keer elk woord voorkomt
counter = Counter()
for text in df_train['text']:
    counter.update(simple_tokenizer(text))

# maak een woordenboek aan
vocab = {word: idx + 2 for idx, (word, _) in enumerate(counter.most_common(MAX_NUM_WORDS))}
vocab['<pad>'] = 0 # extra token voor padding
vocab['<unk>'] = 1 # extra token voor onbekende woorden

print('Size of vocabulary', len(vocab))

# zet elk woord om naar een getal
def encode(text, vocab, max_len):
    tokens = simple_tokenizer(text) # text -> woorden
    idxs = [vocab.get(token, vocab['<unk>']) for token in tokens] # elk woord -> idx
    idxs = idxs[:max_len] # truncating
    idxs += [vocab['<pad>']] * (max_len - len(idxs)) # padding
    return idxs

num_samples = 1000
X_train = torch.tensor([encode(text, vocab, MAX_SEQUENCE_LENGTH) for text in df_train['text'][:num_samples]])
y_train = df_train['label'][:num_samples]
print(X_train.shape)

X_test = torch.tensor([encode(text, vocab, MAX_SEQUENCE_LENGTH) for text in df_test['text'][:num_samples]])
y_test = df_test['label'][:num_samples]
print(X_test.shape)

# Dataset + dataloader
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        super(TextDataset, self).__init__()
        
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

t, l = next(iter(train_loader))

print(t.shape, l.shape)

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


Size of vocabulary 20002
torch.Size([1000, 50])
torch.Size([1000, 50])
torch.Size([64, 50]) torch.Size([64])


/tmp/ipykernel_118/420127414.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


## Opbouwen, trainen en evalueren van een RNN

In [27]:
# RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=5, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self,x):
        x = self.embedding(x)
        outputs, hidden = self.rnn(x)
        # wil je een sequentie als output -> werk met de outputs
        # wil je enkel classificatie doen op de hele tekst -> dan kan je met de hidden werken of je selecteert de laatste output
        x = hidden[0]
        return self.fc(x)

model = RNNModel(len(vocab), EMBEDDING_DIM, 100, 4)

t, l = next(iter(train_loader))
outputs = model(t)

print(t.shape, l.shape)
print(outputs.shape)


torch.Size([64, 50]) torch.Size([64])
torch.Size([64, 4])


/tmp/ipykernel_118/420127414.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


In [28]:
# Train het Model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

num_epochs = 20
model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for texts, labels in train_loader:

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = loss.item()

    print(f"Epoch {epoch +1}/{num_epochs}, loss: {running_loss/len(train_loader)}")

/tmp/ipykernel_118/420127414.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


Epoch 1/20, loss: 0.08051504194736481
Epoch 2/20, loss: 0.08106335997581482
Epoch 3/20, loss: 0.0706050917506218
Epoch 4/20, loss: 0.07702729851007462
Epoch 5/20, loss: 0.07443859428167343
Epoch 6/20, loss: 0.06988861411809921
Epoch 7/20, loss: 0.062111109495162964
Epoch 8/20, loss: 0.051068197935819626
Epoch 9/20, loss: 0.06091461330652237
Epoch 10/20, loss: 0.06228755787014961
Epoch 11/20, loss: 0.042413994669914246
Epoch 12/20, loss: 0.046234361827373505
Epoch 13/20, loss: 0.04114314168691635
Epoch 14/20, loss: 0.05761498212814331
Epoch 15/20, loss: 0.057566843926906586
Epoch 16/20, loss: 0.033861447125673294
Epoch 17/20, loss: 0.0342138446867466
Epoch 18/20, loss: 0.0481715127825737
Epoch 19/20, loss: 0.03751064091920853
Epoch 20/20, loss: 0.036112114787101746


In [29]:
# Evalueer het Model

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predictions = torch.max(outputs, dim=1)

        total += len(labels)
        correct += (predictions == labels).sum().item()

print('total:', total, 'correcte:', correct, 'acc:', correct/total)

total: 1000 correcte: 261 acc: 0.261


/tmp/ipykernel_118/420127414.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


## Oefeningen

* Voeg een extra Dense-laag toe na de RNN-laag. Experimenteer met het aantal neuronen in deze laag en analyseer hoe de prestaties veranderen.
* Pas het model aan om in plaats van een SimpleRNN-laag een LSTM of GRU-laag te gebruiken. Vergelijk de prestaties van de drie typen recurrente netwerken.

In [ ]:
# Oefening 1

In [ ]:
# Oefening 2

**Oefening 3**

Volg de tutorial op de volgende link: https://www.tensorflow.org/text/tutorials/text_generation
Werk hieronder het gelijkaardige probleem uit maar maak het door gebruik te maken van pytorch in plaats van tensorflow voor het model op te bouwen.
In deze tutorial wordt er tekst gegenereerd die lijkt op tekst geschreven door shakespeare.
Let op dat dit een vereenvoudigde versie is waarbij karakter per karakter wordt gegenereerd en niet woord per woord. Er is dus geen garantie dat er echte woorden gemaakt worden.

In [ ]:
import keras
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
import random

path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')
print(text[:250])
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

# Character to index mapping
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for idx, char in enumerate(vocab)}

# TODO: Encodeer elk karakter in tekst naar een nummer, uitkomst is een list ipv een string

# TODO: Maak een dataset aan waarbij de tekst (uit voorgaande todo) omzet naar een reeks sequenties
# input 100 aaneensluitende karakters, output is het karakter erop volgende

# TODO: indien nodig maak een subset tot 10 of 1% van de dataset

# Check a single example
sample_x, sample_y = dataset[0]
print("Input (x):", sample_x)
print("Target (y):", sample_y)
print("Decoded Input:", ''.join(idx_to_char[idx] for idx in sample_x.numpy()))
print("Decoded Target:", idx_to_char[sample_y.item()])
print('Rows', len(dataset))

In [ ]:
# TODO: Maak een rnn model bestaande uit een embedding layer, gru layer en linear layer
# Maak het mogelijk om aan de forward funtie een parameter toe te voegen om ook de hidden state terug te geven en om de hidden state mee te geven voor de gru laag
# 
# TODO: Maak een rnn model bestaande uit een embedding layer, gru layer en linear layer
# Maak het mogelijk om aan de forward funtie een parameter toe te voegen om ook de hidden state terug te geven en om de hidden state mee te geven voor de gru laag
# 
vocab_size = len(idx_to_char)
print(vocab_size)
embedding_dim = 50
rnn_units = 60

In [ ]:
# test 1 sample om door het model te sturen
# kijk of je dimensies correct aan elkaar gekoppeld zijn

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import math

batch_size = 64
seq_length = 100
epochs = 5
vocab_size = len(vocab)
embedding_dim = 50
rnn_units = 60

shakespeare = ShakespeareModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
import torch.nn.functional as F

def generate_text(model, start_string, char_to_idx, idx_to_char, vocab_size, generation_length=100, temperature=1.0):
    model.eval()  # Set model to evaluation mode
    
    # Convert start_string to indices
    input_indices = torch.tensor([char_to_idx[char] for char in start_string], dtype=torch.long).unsqueeze(0)
    
    generated_text = start_string
    states = None  # Initial state (None means it will be initialized automatically)
    
    for _ in range(generation_length):
        # Genereer opeenvolgend nieuwe tokens
        pass
    
    return generated_text

In [ ]:
# Example start string and generation parameters
start_string = "ROMEO: "
generation_length = 200
temperature = 0.8

# Generate text
generated_text = generate_text(
    model=shakespeare,
    start_string=start_string,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    vocab_size=vocab_size,
    generation_length=generation_length,
    temperature=temperature
)

print("Generated Text:")
print(generated_text)
